In [ ]:
# ====================================================================
# 🌌 QEPC UNIVERSAL LOADER
# ====================================================================
import sys, os
# Add the project root (one level up) to the path so we can import the context
sys.path.append(os.path.abspath('..')) 

# Load everything from the central context file
from notebook_context import * # Trigger the success message
print_ready_message()
print("="*80)

-----------------------------------------
---------------------------------------

In [1]:
# --- Corrected Diagnostic Cell ---
qa.qepc_step("Diagnosing Final Team Name Mismatch...")

# 1. Load the newly calculated ORtg/DRtg table (REMOVING reload=True)
calculated_strengths = qa.load_team_stats() 

print("\n--- FIRST 10 TEAM NAMES IN CALCULATED STRENGTHS ---")

# Print the names and the size of the table
print(f"Total teams found: {len(calculated_strengths)}")
print(calculated_strengths['Team'].head(10).tolist())

qa.qepc_step("Diagnosis Complete.")

NameError: name 'qa' is not defined